In [35]:
# RK4 Algorithm is needed

import numpy as np
from pyscf.fci import direct_ep
import direct_ep_cmplx

def evol(ket, tau, t, u, g, hpp, nsite, nelec, nphonon):
    # RK4 
    k1 = -1.0j * direct_ep_cmplx.contract_all(t, u, g, hpp, ket, 
    nsite, nelec, nphonon)
    k2 = -1.0j * direct_ep_cmplx.contract_all(t, u, g, hpp, ket+.5*tau*k1,
    nsite, nelec, nphonon)
    k3 = -1.0j * direct_ep_cmplx.contract_all(t, u, g, hpp, ket+.5*tau*k2,
    nsite, nelec, nphonon)
    k4 = -1.0j * direct_ep_cmplx.contract_all(t, u, g, hpp, ket+1.*tau*k3,
    nsite, nelec, nphonon)

    newket = ket + (1/6)*tau*(k1 + 2.*k2 + 2.*k3 + k4)
    return newket

class HH_Model:

    def __init__(self, t, u, g, omega, nsite, ket=None):

        self.nsite = nsite
        self.nelec = nsite
        self.nphonon = nsite

        self.t = t
        self.u = u
        self.g = g
        self.hpp = np.eye(self.nphonon) * omega
        self.ket = ket
        self.energy = None

    def gs_kernel(self):
        e, ket = direct_ep.kernel(self.t, self.u, self.g, self.hpp, 
        self.nelec, self.nphonon)

        self.energy = e
        self.ket = ket

    def tdfci(self, delta_t, target_t):
        for i in range(int(target_t / delta_t)):
            self.ket = evol(self.ket, delta_t, self.t, self.u, self.g,
            self.hpp, self.nsite, self.nelec, self.nphonon)

    def get_energy(self):
        hc = direct_ep.contract_all(self.t, self.u, self.g, self.hpp,
        self.ket, self.nsite, self.nelec, self.nphonon)
        return np.dot(self.ket.conj().reshape(-1), hc.reshape(-1))

    def get_phonon_energy(self):
        hc = direct_ep_cmplx.contract_pp(self.hpp, self.ket, 
        self.nsite, self.nelec, self.nphonon)
        return np.dot(self.ket.conj().reshape(-1), hc.reshape(-1)) / \
            np.dot(self.ket.conj().reshape(-1), self.ket.reshape(-1))


In [36]:
L = 2
t = np.zeros((L, L))
idx = np.arange(L-1)
t[idx+1,idx] = t[idx,idx+1] = -1
U = 0.
g = 0.
hpp = np.eye(L) * 1.
e, ket = direct_ep.kernel(t, U, g, hpp, L, L, L)
print(e)

t = np.zeros((L, L), dtype=np.complex128)
idx = np.arange(L-1)
t[idx+1,idx] = t[idx,idx+1] = -1+0j

hpp = np.eye(L, dtype=np.complex128) * 1+0j

model = HH_Model(t, 0+0j, 1+0j, hpp, L, ket=ket.astype(np.complex128))


-1.9999999999995075


In [37]:
epplst = []

for i in range(5000):
    phon_e = model.get_phonon_energy()
    print(phon_e.real)

    model.tdfci(delta_t = 5E-4, target_t = 5E-3)
    epplst.append(phon_e.real)

epplst = np.array(epplst)
np.savetxt("epplst.txt", epplst)

4.664252393019408e-13
7.499944972694135e-05
0.00029999092001894924
0.0006749537639305663
0.0011998535193372547
0.001874641834882767
0.0026992563659282544
0.0036736206412445738
0.004797643900800371
0.006071220905075226
0.007494231716403007
0.009066541452925075
0.010788000015806544
0.012658441790440518
0.014677685322435569
0.01684553296925028
0.01916177052840507
0.02162616684326621
0.024238473387459054
0.026998423829027448
0.029905733575513616
0.03296009930118784
0.03616119845770881
0.03950868876954511
0.04300220771553446
0.04664137199799974
0.05042577700088212
0.05435499623838658
0.05842858079566982
0.06264605876312973
0.06700693466588203
0.0715106888900325
0.07615677710737309
0.08094462970014447
0.0858736511875209
0.09094321965548025
0.09615268619172758
0.10150137432734038
0.10698857948680124
0.11261356844807646
0.11837557881438909
0.1242738184993202
0.1303074652268545
0.13647566604796513
0.14277753687530673
0.1492121620375579
0.15577859385492218
0.16247585223725922
0.1693029243062824


In [39]:
import matplotlib.pyplot as plt
import numpy as np

epplst = np.loadtxt("epplst.txt")

plt.plot(np.arange(5000) * 5E-3, epplst)

In [8]:
# DMRG

import numpy as np
from itertools import accumulate
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

L = 2
omega = 1.0
g = 1.
nbasis_fermion = L
nbasis_boson = L
nbcuts = [2] * L

driver = DMRGDriver(scratch="./tmp", symm_type=(SymmetryTypes.SZ|SymmetryTypes.CPX), n_threads=4)
driver.initialize_system(n_sites=nbasis_fermion+nbasis_boson, n_elec=L, spin=0, orb_sym=None)

GH = driver.bw.bs.GeneralHamiltonian

class HubbardHamiltonian(GH):

    def __init__(self, driver, vacuum, n_sites, orb_sym, n_sites_fermion = 0, n_sites_boson = 0, nbcuts = []):
        GH.__init__(self)
        self.driver = driver
        self.opf = driver.bw.bs.OperatorFunctions(driver.bw.brs.CG())
        self.vacuum = vacuum
        self.n_sites = n_sites
        self.n_sites_fermion = n_sites_fermion
        self.n_sites_boson = n_sites_boson
        self.nbcuts = nbcuts
        self.orb_sym = orb_sym

        # Add bases for Boson [Shuoxue]
        basis_fermion = [self.get_site_basis(m) for m in range(self.n_sites_fermion)]
        basis_boson = [self.get_site_basis_boson(self.nbcuts[k]) for k in range(self.n_sites_boson)]
        self.basis = self.driver.bw.brs.VectorStateInfo(basis_fermion + basis_boson)
        
        self.site_op_infos = self.driver.bw.brs.VectorVectorPLMatInfo([
            self.driver.bw.brs.VectorPLMatInfo() for _ in range(self.n_sites)
        ])
        self.site_norm_ops = self.driver.bw.bs.VectorMapStrSpMat([
            self.driver.bw.bs.MapStrSpMat() for _ in range(self.n_sites)
        ])
        self.init_site_ops()

    def get_site_basis(self, m):
        """Single site states."""
        bz = self.driver.bw.brs.StateInfo()
        bz.allocate(4)
        bz.quanta[0] = self.driver.bw.SX(0, 0, 0)
        bz.quanta[1] = self.driver.bw.SX(1, 1, self.orb_sym[m])
        bz.quanta[2] = self.driver.bw.SX(1, -1, self.orb_sym[m])
        bz.quanta[3] = self.driver.bw.SX(2, 0, 0)
        bz.n_states[0] = bz.n_states[1] = bz.n_states[2] = bz.n_states[3] = 1
        bz.sort_states()
        return bz

    # Site basis for Boson [Shuoxue]
    def get_site_basis_boson(self, nbcut):
        """Single-site Boson states with nbcut"""

        bz = self.driver.bw.brs.StateInfo()
        bz.allocate(1)
        bz.quanta[0] = self.driver.bw.SX(0,0,0)
        bz.n_states[0] = nbcut  # add the cutoff
        bz.sort_states()
        return bz

    def init_site_ops(self):
        """Initialize operator quantum numbers at each site (site_op_infos)
        and primitive (single character) site operators (site_norm_ops)."""
        i_alloc = self.driver.bw.b.IntVectorAllocator()
        d_alloc = self.driver.bw.b.DoubleVectorAllocator()
        # site op infos
        max_n, max_s = 10, 10
        max_n_odd, max_s_odd = max_n | 1, max_s | 1
        max_n_even, max_s_even = max_n_odd ^ 1, max_s_odd ^ 1
        for m in range(self.n_sites_fermion):
            qs = {self.vacuum}
            for n in range(-max_n_odd, max_n_odd + 1, 2):
                for s in range(-max_s_odd, max_s_odd + 1, 2):
                    qs.add(self.driver.bw.SX(n, s, self.orb_sym[m]))
            for n in range(-max_n_even, max_n_even + 1, 2):
                for s in range(-max_s_even, max_s_even + 1, 2):
                    qs.add(self.driver.bw.SX(n, s, 0))
            for q in sorted(qs):
                mat = self.driver.bw.brs.SparseMatrixInfo(i_alloc)
                mat.initialize(self.basis[m], self.basis[m], q, q.is_fermion)
                self.site_op_infos[m].append((q, mat))

        # Initialize site_op_infos for Boson [Shuoxue]
        for k in range(self.n_sites_boson):
            mat = self.driver.bw.brs.SparseMatrixInfo(i_alloc)
            mat.initialize(self.basis[k+self.n_sites_fermion], self.basis[k+self.n_sites_fermion], self.vacuum, self.vacuum.is_fermion)
            self.site_op_infos[k + self.n_sites_fermion].append((self.vacuum, mat))
        
        # prim ops
        for m in range(self.n_sites_fermion):

            # ident
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(m, self.driver.bw.SX(0, 0, 0))
            mat.allocate(info)
            mat[info.find_state(self.driver.bw.SX(0, 0, 0))] = np.array([1.0])
            mat[info.find_state(self.driver.bw.SX(1, 1, self.orb_sym[m]))] = np.array([1.0])
            mat[info.find_state(self.driver.bw.SX(1, -1, self.orb_sym[m]))] = np.array([1.0])
            mat[info.find_state(self.driver.bw.SX(2, 0, 0))] = np.array([1.0])
            self.site_norm_ops[m][""] = mat

            # C alpha
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(m, self.driver.bw.SX(1, 1, self.orb_sym[m]))
            mat.allocate(info)
            mat[info.find_state(self.driver.bw.SX(0, 0, 0))] = np.array([1.0])
            mat[info.find_state(self.driver.bw.SX(1, -1, self.orb_sym[m]))] = np.array([1.0])
            self.site_norm_ops[m]["c"] = mat

            # D alpha
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(m, self.driver.bw.SX(-1, -1, self.orb_sym[m]))
            mat.allocate(info)
            mat[info.find_state(self.driver.bw.SX(1, 1, self.orb_sym[m]))] = np.array([1.0])
            mat[info.find_state(self.driver.bw.SX(2, 0, 0))] = np.array([1.0])
            self.site_norm_ops[m]["d"] = mat

            # C beta
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(m, self.driver.bw.SX(1, -1, self.orb_sym[m]))
            mat.allocate(info)
            mat[info.find_state(self.driver.bw.SX(0, 0, 0))] = np.array([1.0])
            mat[info.find_state(self.driver.bw.SX(1, 1, self.orb_sym[m]))] = np.array([-1.0])
            self.site_norm_ops[m]["C"] = mat

            # D beta
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(m, self.driver.bw.SX(-1, 1, self.orb_sym[m]))
            mat.allocate(info)
            mat[info.find_state(self.driver.bw.SX(1, -1, self.orb_sym[m]))] = np.array([1.0])
            mat[info.find_state(self.driver.bw.SX(2, 0, 0))] = np.array([-1.0])
            self.site_norm_ops[m]["D"] = mat

            # Nup * Ndn
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(m, self.driver.bw.SX(0, 0, 0))
            mat.allocate(info)
            mat[info.find_state(self.driver.bw.SX(2, 0, 0))] = np.array([1.0])
            self.site_norm_ops[m]["N"] = mat

        for k in range(self.n_sites_boson):

            vac = self.driver.bw.SX(0, 0, 0)

            # Identity 
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(k+self.n_sites_fermion, vac)
            mat.allocate(info)
            mat[info.find_state(vac)] = np.eye(self.nbcuts[k])
            self.site_norm_ops[k + self.n_sites_fermion][""] = mat

            # b^\dagger as E
            op_bdagger = np.zeros((self.nbcuts[k], self.nbcuts[k]))
            for i in range(self.nbcuts[k] - 1):
                op_bdagger[i+1, i] = np.sqrt(i+1)
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(k+self.n_sites_fermion, vac)
            mat.allocate(info)
            mat[info.find_state(vac)] = op_bdagger
            self.site_norm_ops[k + self.n_sites_fermion]["E"] = mat

            # b as F
            op_b = np.zeros((self.nbcuts[k], self.nbcuts[k]))
            for i in range(self.nbcuts[k] - 1):
                op_b[i, i+1] = np.sqrt(i+1)
            mat = self.driver.bw.bs.SparseMatrix(d_alloc)
            info = self.find_site_op_info(k+self.n_sites_fermion, vac)
            mat.allocate(info)
            mat[info.find_state(vac)] = op_b
            self.site_norm_ops[k + self.n_sites_fermion]["F"] = mat

    def get_site_string_ops(self, m, ops):
        """Construct longer site operators from primitive ones."""
        d_alloc = self.driver.bw.b.DoubleVectorAllocator()
        for k in ops:
            if k in self.site_norm_ops[m]:
                ops[k] = self.site_norm_ops[m][k]
            else:
                xx = self.site_norm_ops[m][k[0]]
                for p in k[1:]:
                    xp = self.site_norm_ops[m][p]
                    q = xx.info.delta_quantum + xp.info.delta_quantum
                    mat = self.driver.bw.bs.SparseMatrix(d_alloc)
                    mat.allocate(self.find_site_op_info(m, q))
                    self.opf.product(0, xx, xp, mat)
                    xx = mat
                ops[k] = self.site_norm_ops[m][k] = xx
        return ops

    def init_string_quanta(self, exprs, term_l, left_vacuum):
        """Quantum number for string operators (orbital independent part)."""
        qs = {
            'N': self.driver.bw.SX(0, 0, 0),
            'c':  self.driver.bw.SX(1, 1, 0),
            'C':  self.driver.bw.SX(1, -1, 0),
            'd':  self.driver.bw.SX(-1, -1, 0),
            'D':  self.driver.bw.SX(-1, 1, 0),
            "E": self.driver.bw.SX(0, 0, 0),
            "F": self.driver.bw.SX(0, 0, 0)
        }
        return self.driver.bw.VectorVectorSX([self.driver.bw.VectorSX(list(accumulate(
            [qs['N']] + [qs[x] for x in expr], lambda x, y: x + y)))
            for expr in exprs
        ])

    def get_string_quanta(self, ref, expr, idxs, k):
        """Quantum number for string operators (orbital dependent part)."""
        l, r = ref[k], ref[-1] - ref[k]
        for j, (ex, ix) in enumerate(zip(expr, idxs)):
            ipg = self.orb_sym[ix]
            if ex == "N":
                pass
            elif j < k:
                l.pg = l.pg ^ ipg
            else:
                r.pg = r.pg ^ ipg
        return l, r

    def get_string_quantum(self, expr, idxs):
        """Total quantum number for a string operator."""
        qs = lambda ix: {
            'N': self.driver.bw.SX(0, 0, 0),
            'c':  self.driver.bw.SX(1, 1, self.orb_sym[ix]),
            'C':  self.driver.bw.SX(1, -1, self.orb_sym[ix]),
            'd':  self.driver.bw.SX(-1, -1, self.orb_sym[ix]),
            'D':  self.driver.bw.SX(-1, 1, self.orb_sym[ix]),
            "E": self.driver.bw.SX(0, 0, 0),
            "F": self.driver.bw.SX(0, 0, 0)
        }
        return sum([qs(0)['N']] + [qs(ix)[ex] for ex, ix in zip(expr, idxs)])

    def deallocate(self):
        """Release memory."""
        for ops in self.site_norm_ops:
            for p in ops.values():
                p.deallocate()
        for infos in self.site_op_infos:
            for _, p in infos:
                p.deallocate()
        for bz in self.basis:
            bz.deallocate()

driver.ghamil = HubbardHamiltonian(driver, driver.vacuum, driver.n_sites, 
    driver.orb_sym, n_sites_fermion=nbasis_fermion, n_sites_boson=nbasis_boson, nbcuts=nbcuts)

In [9]:
b = driver.expr_builder()

for t, ops in zip([-1, 1, -1, 1], ["cd", "dc", "CD", "DC"]):
    b.add_term(ops, np.array([[i, i + 1] for i in range(L - 1)]).flatten(),
        [t] * (L - 1))
        
b.add_term("EF", np.array([[i, i] for i in range(nbasis_fermion, nbasis_boson + nbasis_fermion)]).flatten(), [omega] * L)

finilized_b = b.finalize(adjust_order=True)

print(finilized_b)

mpo = driver.get_mpo(finilized_b, iprint=2)
ket = driver.get_random_mps(tag="GS", bond_dim=250, nroots=1)

def run_dmrg(driver, mpo):
    bond_dims = [250] * 4 + [500] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(
        mpo,
        ket,
        n_sweeps=10,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=2,
    )

energies = run_dmrg(driver, mpo)
print(energies)

ket_evv = ket

 SU2 = 0 CONST E = (0.0000000000000000,0.0000000000000000)
 TERM cd ::
      0      1 = (-1.0000000000000000,0.0000000000000000)
 TERM dc ::
      0      1 = (1.0000000000000000,0.0000000000000000)
 TERM CD ::
      0      1 = (-1.0000000000000000,0.0000000000000000)
 TERM DC ::
      0      1 = (1.0000000000000000,0.0000000000000000)
 TERM EF ::
      2      2 = (1.0000000000000000,0.0000000000000000)
      3      3 = (1.0000000000000000,0.0000000000000000)


Build MPO | Nsites =     4 | Nterms =          6 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     4 .. Mmpo =     5 DW = 0.00e+00 NNZ =        5 SPT = 0.0000 Tmvc = 0.001 T = 0.016
 Site =     1 /     4 .. Mmpo =     2 DW = 0.00e+00 NNZ =        5 SPT = 0.5000 Tmvc = 0.000 T = 0.003
 Site =     2 /     4 .. Mmpo =     2 DW = 0.00e+00 NNZ =        3 SPT = 0.2500 Tmvc = 0.000 T = 0.007
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        2 SPT = 0.0000 Tmvc = 0.001 T = 0.009
Ttotal =      0.036 Tmvc-total =

In [10]:
# Phonon energy MPO
b = driver.expr_builder()

b.add_term("EF", np.array([[i, i] for i in range(nbasis_fermion, nbasis_boson + nbasis_fermion)]).flatten(), [omega] * L)

finilized_b = b.finalize(adjust_order=True)
print(finilized_b)
hpp = driver.get_mpo(finilized_b, iprint=2)


 SU2 = 0 CONST E = (0.0000000000000000,0.0000000000000000)
 TERM EF ::
      2      2 = (1.0000000000000000,0.0000000000000000)
      3      3 = (1.0000000000000000,0.0000000000000000)


Build MPO | Nsites =     4 | Nterms =          2 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        1 SPT = 0.0000 Tmvc = 0.000 T = 0.003
 Site =     1 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        1 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     2 /     4 .. Mmpo =     2 DW = 0.00e+00 NNZ =        2 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        2 SPT = 0.0000 Tmvc = 0.000 T = 0.002
Ttotal =      0.009 Tmvc-total = 0.000 MPO bond dimension =     2 MaxDW = 0.00e+00
NNZ =            6 SIZE =            6 SPT = 0.0000

Rank =     0 Ttotal =      0.026 MPO method = FastBipartite bond dimension =       2 NNZ =            6 SIZE =            6 SPT = 0.0000


In [11]:
# Time Evolving MPO
b = driver.expr_builder()

for t, ops in zip([-1, 1, -1, 1], ["cd", "dc", "CD", "DC"]):
    b.add_term(ops, np.array([[i, i + 1] for i in range(L - 1)]).flatten(),
        [t] * (L - 1))
        
for t, ops in zip([g, g, g, g], ["cdE", "CDE", "cdF", "CDF"]):
    b.add_term(ops, np.array([[i, i, i + nbasis_fermion] for i in range(L)]).flatten(), [t] * L)

b.add_term("EF", np.array([[i, i] for i in range(nbasis_fermion, nbasis_boson + nbasis_fermion)]).flatten(), [omega] * L)

finilized_b = b.finalize(adjust_order=True)

print(finilized_b)

mpo = driver.get_mpo(finilized_b, iprint=2)


 SU2 = 0 CONST E = (0.0000000000000000,0.0000000000000000)
 TERM cd ::
      0      1 = (-1.0000000000000000,0.0000000000000000)
 TERM dc ::
      0      1 = (1.0000000000000000,0.0000000000000000)
 TERM CD ::
      0      1 = (-1.0000000000000000,0.0000000000000000)
 TERM DC ::
      0      1 = (1.0000000000000000,0.0000000000000000)
 TERM cdE ::
      0      0      2 = (1.0000000000000000,0.0000000000000000)
      1      1      3 = (1.0000000000000000,0.0000000000000000)
 TERM CDE ::
      0      0      2 = (1.0000000000000000,0.0000000000000000)
      1      1      3 = (1.0000000000000000,0.0000000000000000)
 TERM cdF ::
      0      0      2 = (1.0000000000000000,0.0000000000000000)
      1      1      3 = (1.0000000000000000,0.0000000000000000)
 TERM CDF ::
      0      0      2 = (1.0000000000000000,0.0000000000000000)
      1      1      3 = (1.0000000000000000,0.0000000000000000)
 TERM EF ::
      2      2 = (1.0000000000000000,0.0000000000000000)
      3      3 = (1.0000000000

In [12]:
ephlst = []

impo = driver.get_identity_mpo()

for a in range(5000):
    eph = driver.expectation(ket_evv, hpp, ket_evv)
    norm = driver.expectation(ket_evv, impo, ket_evv)
    eph = eph / norm.real
    ephlst.append(eph.real)
    print(eph.real)
    ket_evv = driver.td_dmrg(mpo, ket_evv, delta_t=5E-4j, target_t=5E-3j, iprint=0, final_mps_tag="TD{}".format(a%2)
    )

np.savetxt("epplst_dmrg.txt", ephlst)

6.844449166929985e-12
7.500658913491257e-05
0.00029997893175442997
0.0006748289296527865
0.0011994098032206341
0.0018735161853134937
0.0026968842429817563
0.00366919183380678
0.0047900586967084356
0.006059046677071634
0.007475659986007283
0.009039345493492026
0.010749493055252758
0.012605435872963838
0.014606450887582004
0.0167517592055626
0.01904052655713405
0.021471863787077248
0.02404482737699527
0.02675841999879574
0.029611591097940922
0.03260323750958545
0.03573220410063975
0.038997284445610396
0.04239722152681226
0.0459307084639493
0.04959638927537418
0.053392859658862424
0.05731866780604727
0.06137231523133523
0.06555225764580541
0.0698569058294472
0.07428462655103517
0.07883374349743337
0.08350253821474625
0.08828925109630241
0.09319208238642013
0.09820919317918926
0.10333870645888535
0.10857870814749909
0.11392724817475934
0.11938234158089256
0.12494196958887457
0.1306040807754765
0.1363665920764369
0.14222739009545454
0.14818433214097224
0.1542352474196687
0.1603779382433122


# Holstein Model

$$
\hat H = \hat H_{hop} + \omega \sum\limits_{i} \hat b^\dagger_i \hat b_i +g \sum\limits_{i} \hat n_i (\hat b^\dagger_i + \hat  b_i)
$$

$$
g = \begin{cases}0, \tau < 0; \\ t, \tau > 0.\end{cases}
$$

2-site, 1 phonon

Bond-dimension for DMRG is 200.

Evaluate the Phonon energy . RK4 algorithm, and the time $\delta \tau = 5 \times 10^{-4} t $ for both FCI and DMRG.

May have further problems of TD-DMRG code? Or FCI block goes wrong?

In [13]:
# Unnormalized
import matplotlib.pyplot as plt
epplst = np.loadtxt("epplst.txt")

plt.plot(epplst, label='FCI')
plt.plot(ephlst, label='DMRG') # Bond Dimension 200

plt.legend()
plt.xlabel(r"$t / (5\times 10^{-3})$ a.u." )
plt.ylabel(r"$E_{ph}$")

Text(0, 0.5, '$E_{ph}$')

In [49]:
# Normalized
# DMRG results collapses to ground state of g = t.

epplst = np.loadtxt("epplst.txt")

plt.plot(np.arange(5000) * 5E-3, epplst, label='FCI')
plt.plot(np.arange(5000) * 5E-3, ephlst, label='DMRG') # Bond Dimension 200

plt.hlines(0.47130458536081793, 0, 5E-3*5000, colors='black', label=r'$E_{gs}$')

plt.legend()
plt.xlabel(r"$t / (5\times 10^{-3})$ a.u." )
plt.ylabel(r"$E_{ph}$")

Text(0, 0.5, '$E_{ph}$')

In [46]:
# Time Evolving MPO
b = driver.expr_builder()

for t, ops in zip([-1, 1, -1, 1], ["cd", "dc", "CD", "DC"]):
    b.add_term(ops, np.array([[i, i + 1] for i in range(L - 1)]).flatten(),
        [t] * (L - 1))
        
for t, ops in zip([1, 1, 1, 1], ["cdE", "CDE", "cdF", "CDF"]):
    b.add_term(ops, np.array([[i, i, i + nbasis_fermion] for i in range(L)]).flatten(), [t] * L)

b.add_term("EF", np.array([[i, i] for i in range(nbasis_fermion, nbasis_boson + nbasis_fermion)]).flatten(), [omega] * L)

finilized_b = b.finalize(adjust_order=True)

print(finilized_b)

mpo = driver.get_mpo(finilized_b, iprint=2)

ket2 = driver.get_random_mps(tag="GS", bond_dim=250, nroots=1)

def run_dmrg(driver, mpo):
    bond_dims = [250] * 4 + [500] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(
        mpo,
        ket2,
        n_sweeps=10,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=2,
    )

energies = run_dmrg(driver, mpo)

 SU2 = 0 CONST E = (0.0000000000000000,0.0000000000000000)
 TERM cd ::
      0      1 = (-1.0000000000000000,0.0000000000000000)
 TERM dc ::
      0      1 = (1.0000000000000000,0.0000000000000000)
 TERM CD ::
      0      1 = (-1.0000000000000000,0.0000000000000000)
 TERM DC ::
      0      1 = (1.0000000000000000,0.0000000000000000)
 TERM cdE ::
      0      0      2 = (1.0000000000000000,0.0000000000000000)
      1      1      3 = (1.0000000000000000,0.0000000000000000)
 TERM CDE ::
      0      0      2 = (1.0000000000000000,0.0000000000000000)
      1      1      3 = (1.0000000000000000,0.0000000000000000)
 TERM cdF ::
      0      0      2 = (1.0000000000000000,0.0000000000000000)
      1      1      3 = (1.0000000000000000,0.0000000000000000)
 TERM CDF ::
      0      0      2 = (1.0000000000000000,0.0000000000000000)
      1      1      3 = (1.0000000000000000,0.0000000000000000)
 TERM EF ::
      2      2 = (1.0000000000000000,0.0000000000000000)
      3      3 = (1.0000000000

In [47]:
# Phonon energy MPO
b = driver.expr_builder()

b.add_term("EF", np.array([[i, i] for i in range(nbasis_fermion, nbasis_boson + nbasis_fermion)]).flatten(), [omega] * L)

finilized_b = b.finalize(adjust_order=True)
print(finilized_b)
hpp = driver.get_mpo(finilized_b, iprint=2)

print(driver.expectation(ket2, hpp, ket2))

 SU2 = 0 CONST E = (0.0000000000000000,0.0000000000000000)
 TERM EF ::
      2      2 = (1.0000000000000000,0.0000000000000000)
      3      3 = (1.0000000000000000,0.0000000000000000)


Build MPO | Nsites =     4 | Nterms =          2 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        1 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        1 SPT = 0.0000 Tmvc = 0.000 T = 0.003
 Site =     2 /     4 .. Mmpo =     2 DW = 0.00e+00 NNZ =        2 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        2 SPT = 0.0000 Tmvc = 0.000 T = 0.003
Ttotal =      0.010 Tmvc-total = 0.000 MPO bond dimension =     2 MaxDW = 0.00e+00
NNZ =            6 SIZE =            6 SPT = 0.0000

Rank =     0 Ttotal =      0.021 MPO method = FastBipartite bond dimension =       2 NNZ =            6 SIZE =            6 SPT = 0.0000
(0.47130458536081793-3.38424535201680